In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import qcsapphire
import numpy as np
import time

In [3]:
qcsapphire.discover_devices()

[['COM3',
  'Intel(R) Active Management Technology - SOL (COM3)',
  'PCI\\VEN_8086&DEV_43E3&SUBSYS_0A541028&REV_11\\3&11583659&1&B3'],
 ['COM4',
  'USB Serial Device (COM4)',
  'USB VID:PID=04D8:000A SER= LOCATION=1-10:x.0'],
 ['COM5',
  'USB Serial Device (COM5)',
  'USB VID:PID=0483:A3E5 SER=206A36705430 LOCATION=1-9:x.0'],
 ['COM7',
  'USB Serial Device (COM7)',
  'USB VID:PID=239A:8014 SER=3B0D07C25831555020312E341A3214FF LOCATION=1-5:x.0']]

In [4]:
pulser = qcsapphire.Pulser('COM4')

In [59]:
def reset_pulser(num_resets = 2):
    for i in range(num_resets):
        pulser.channel('B').state(0)
        pulser.channel('A').state(0)

        pulser.channel('C').state(0)

        pulser.channel('D').state(0)
        pulser.system.state(0)

        pulser.query('*RST')
        pulser.system.mode('normal')
        time.sleep(1)
        
def set_pulse_state_for_rabi(width_chB, 
                             cycle_time = 15e-6, 
                             width_chA = 3e-6, 
                             period = 200e-9, 
                             aom_response_time = 800e-9,
                             rf_response_time = 200e-9,
                             t_pad_1 = 200e-9,
                             trigger_width = 1e-6,
                             rf_pulse_justify = 'left',
                             t_pad_2 = 200e-9):
    
    assert rf_pulse_justify in ['left', 'center', 'right']
    
    reset_pulser()
    pulser.system.period(period)
    
    width_chA = np.round(width_chA,8)
    on_count_chA = 1
    off_count_chA = np.round(cycle_time/period,8).astype(int) - on_count_chA
    channel = pulser.channel('A')
    channel.mode('dcycle')
    channel.width(width_chA)
    channel.pcounter(on_count_chA)
    channel.ocounter(off_count_chA)

    width_chB = np.round(width_chB,8)
    
    if rf_pulse_justify == 'center':
        delay_chB = width_chA + (cycle_time - width_chA)/2 - width_chB/2 - rf_response_time
    if rf_pulse_justify == 'left':
        delay_chB = width_chA + aom_response_time + t_pad_1 - rf_response_time
    if rf_pulse_justify == 'right':
        delay_chB = cycle_time - t_pad_2 - width_chB - rf_response_time + aom_response_time
    
    #todo: check to be sure the RF pulse is fully outside of the aom response + pad time, raise exception if violated

    delay_chB = np.round(delay_chB,8)
    on_count_chB = 1
    off_count_chB = np.round(2*cycle_time/period).astype(int) - on_count_chB
    channel = pulser.channel('B')
    channel.mode('dcycle')
    channel.width(width_chB)
    channel.delay(delay_chB)
    channel.pcounter(on_count_chB)
    channel.ocounter(off_count_chB)

    
    channel = pulser.channel('C')
    channel.mode('normal')
    channel.width(period/2)
    channel.delay(0)

    channel = pulser.channel('D')
    channel.mode('dcycle')
    channel.width(np.round(trigger_width,8))
    channel.delay(0)
    channel.pcounter(1)
    channel.ocounter(np.round(2*cycle_time/period).astype(int) - 1)
    
    pulser.channel('A').state(1)
    pulser.channel('B').state(1)
    pulser.channel('C').state(1)
    pulser.channel('D').state(1)
    pulser.system.state(1)

In [61]:
max_width = 10e-6
aom_width = 3e-6
full_cycle_width = max_width + aom_width + 2e-6
for w in np.arange(.5e-6, max_width, 5e-7):
    w_round = np.round(w,8)
    print(w_round)
    set_pulse_state_for_rabi(w_round, cycle_time = full_cycle_width, center_rf_pulse=True)
    time.sleep(5)
    
# this doesn't work. The QCSapphire system is unstable. It seems that a full reset has to be done

5e-07
1e-06
1.5e-06
2e-06
2.5e-06
3e-06
3.5e-06
4e-06
4.5e-06
5e-06
5.5e-06
6e-06
6.5e-06
7e-06
7.5e-06
8e-06
8.5e-06
9e-06
9.5e-06
1e-05
